# Real estate price model workflow

Sam Maurer, Feb 2018  
Python 3.6, intended to be backward compatible with 2.7

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd

import orca

In [2]:
# Import scripts from this repository
import os; os.chdir('../')

from extensions import modelmanager as mm
from models import RegressionStep

## Bootstrap Orca with some legacy registrations

This exercise starts from a point where data is already registered in Orca. Eventually, the vision is that data will be loaded based on config files in the 'data' directory.

For now, the 'legacy' directory contains some code from Paul Sohn's [urbansim_parcels](https://github.com/urbansim/urbansim_parcels) project. Importing 'datasources.py' and 'models.py' registers a handful of Orca objects.

In [3]:
os.chdir('legacy')
import legacy_datasources
import legacy_models

## Explore the Orca registrations

In [4]:
orca.list_tables()

['households', 'buildings', 'parcels', 'jobs']

In [5]:
orca.list_columns()

[('households', 'node_id'), ('buildings', 'node_id'), ('jobs', 'node_id')]

In [6]:
orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'households'),
 ('buildings', 'jobs'),
 ('nodes', 'buildings')]

In [7]:
orca.list_injectables()

['settings', 'store', 'net_store']

In [8]:
orca.list_steps()

['build_networks', 'neighborhood_vars']

## Explore the data

Orca doesn't execute code to load the registered objects until it needs to.

(Note that there's a problem in the data: only about 2,000 buildings have a 'node_id' linking them to the network aggregations table. This is going to affect the network calculations, and also limit our ability to estimate models.)

In [9]:
orca.get_table('buildings').to_frame().describe()

,parcel_id,development_type_id,improvement_value,residential_units,non_residential_sqft,stories,year_built,residential_sqft,res_price_per_sqft,node_id
count,5.917200e+04,59172.000000,5.917200e+04,59172.000000,5.917200e+04,59172.000000,59172.000000,5.917200e+04,59172.000000,2182.000000
mean,1.824953e+06,19.335733,5.938045e+05,3.662661,8.495205e+03,1.081170,1979.705486,1.303178e+04,214.760125,44284.095325
std,2.138605e+06,0.663925,2.823604e+06,13.734032,4.363458e+04,0.381794,20.675804,4.544811e+04,317.965487,4104.251504
min,2.900000e+01,19.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,1913.000000,5.000000e+01,0.000220,36360.000000
25%,2.894475e+05,19.000000,1.071312e+05,1.000000,0.000000e+00,1.000000,1965.000000,1.472000e+03,133.098672,41131.000000
50%,5.873640e+05,19.000000,1.750000e+05,1.000000,0.000000e+00,1.000000,1981.000000,2.122000e+03,191.676955,43936.500000
75%,5.032626e+06,19.000000,3.022590e+05,1.000000,0.000000e+00,1.000000,1998.000000,3.601000e+03,258.776213,47438.000000
max,5.293696e+06,21.000000,1.800000e+08,679.000000,1.197079e+06,29.000000,2012.000000,1.197079e+06,21770.833333,52513.000000


In [10]:
print(len(orca.get_table('buildings').local_columns))  # native columns only
print(len(orca.get_table('buildings').to_frame().columns))  # native plus virtual

10
11


## Generate accessibility measures for the price model

The network accessibility metrics are not stored on disk; for now we'll generate them using legacy code.

In [11]:
orca.run(['build_networks'])

Running step 'build_networks'
Time to execute step 'build_networks': 0.30 s
Total time to execute iteration 1 with iteration value None: 0.30 s


In [12]:
%%capture
orca.run(['neighborhood_vars'])

In [13]:
orca.list_tables()

['households', 'buildings', 'parcels', 'jobs', 'nodes']

In [14]:
print(orca.get_table('nodes').to_frame().columns.tolist())

['ave_parcel_size', 'jobs_1500m', 'jobs_800m', 'jobs_400m', 'ave_income', 'ave_age_of_head_1500m', 'ave_children_1500m', 'ave_year_built_1500m', 'population_400m', 'jobs_3000m', 'households_3000m', 'residential_units_3000m', 'residential_units_1500m', 'residential_units_800m']


## Display all the registered data columns

In [15]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['building_id', 'tenure', 'persons', 'workers', 'age_of_head', 'income', 'children', 'race_id', 'cars', 'base_luz', 'segmentation_col', 'node_id']

BUILDINGS
['parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'non_residential_sqft', 'stories', 'year_built', 'residential_sqft', 'note', 'res_price_per_sqft', 'node_id']

PARCELS
['parcel_acres', 'development_type_id', 'distance_to_park', 'zoning_id', 'y', 'node_id', 'distance_to_school', 'distance_to_freeway', 'distance_to_onramp', 'mgra_id', 'proportion_undevelopable', 'taz_id', 'distance_to_transit', 'tax_exempt', 'distance_to_coast', 'luz_id', 'land_value', 'county_id', 'msa_id', 'x', 'acres']

JOBS
['sector_id', 'building_id', 'node_id']

NODES
['ave_parcel_size', 'jobs_1500m', 'jobs_800m', 'jobs_400m', 'ave_income', 'ave_age_of_head_1500m', 'ave_children_1500m', 'ave_year_built_1500m', 'population_400m', 'jobs_3000m', 'households_3000m', 'residential_units_3000m', 'residential_units_1500m', 'res

In [16]:
# These are the tables with direct relational links

orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'households'),
 ('buildings', 'jobs'),
 ('nodes', 'buildings')]

# Estimate a price model

The basic idea of the parcel template is that we create model steps by _passing arguments to classes_ rather than by writing Python functions and giving them Orca decorators, as we would for a fully custom model.

Much of the functionality for this is already built into UrbanSim and Orca, we'll just need to extend things here and there.

This demo uses a new RegressionStep() class that provides a full model development workflow: estimating a model, registering it with Orca, saving it for future use.

### Specify parameters and pass them to a model object

In [17]:
# Specify the model expression and names of tables to draw data from (the first table
# is the primary one; additional tables must be able to merge onto it unambiguously)

tables = ['buildings', 'parcels', 'nodes']

model_expression = (
    "np.log1p(res_price_per_sqft) ~ "
        "parcel_acres + "
        "year_built + "
        "ave_income + "
        "distance_to_freeway + "
        "population_400m + "
        "jobs_3000m")

# For prediction, specify destination column (if different from the dependent variable
# used for estimation), and how to reverse the left-hand-side transformation

out_fname = 'fitted_price'
ytransform = np.exp

In [38]:
# Generate a new column to store the fitted prices

zeros = np.repeat(0.0, len(orca.get_table('buildings')))
orca.get_table('buildings').update_col('fitted_price', zeros)

In [19]:
# Create the model object

model = RegressionStep(model_expression, tables, 
                       out_fname=out_fname, ytransform=ytransform)

### Fit the model

In [20]:
model.fit()

                                 OLS Regression Results                                 
Dep. Variable:     np.log1p(res_price_per_sqft)   R-squared:                       0.481
Model:                                      OLS   Adj. R-squared:                  0.479
Method:                           Least Squares   F-statistic:                     335.3
Date:                          Sun, 11 Feb 2018   Prob (F-statistic):          6.74e-305
Time:                                  19:05:47   Log-Likelihood:                -2702.5
No. Observations:                          2182   AIC:                             5419.
Df Residuals:                              2175   BIC:                             5459.
Df Model:                                     6                                         
Covariance Type:                      nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------

### If we like it, register it as an Orca step

In [49]:
model.register()

In [50]:
orca.list_steps()

['build_networks',
 'neighborhood_vars',
 'RegressionStep-20180211-190547',
 'RegressionStep-20180211-190813']

### Run the model step

In [51]:
orca.run(['RegressionStep-20180211-190547'])

Running step 'RegressionStep-20180211-190547'
Predicted 2182 values
Time to execute step 'RegressionStep-20180211-190547': 0.05 s
Total time to execute iteration 1 with iteration value None: 0.06 s


### Check the fitted values

In [52]:
df = orca.get_table('buildings').to_frame(['res_price_per_sqft', 'fitted_price'])
df.loc[df.fitted_price > 0].describe()

,res_price_per_sqft,fitted_price
count,2182.000000,2182.000000
mean,243.849936,206.818870
std,162.403472,137.454712
min,0.974158,0.007513
25%,128.734794,89.025805
50%,228.539201,191.793083
75%,365.494447,278.324228
max,1125.000000,1484.329439


### BONUS

Running "model.register()" also registered the step with the new ModelManager extention, which saved it to disk so that it can be automatically re-loaded in the future.

In [21]:
d = model.to_dict()

In [22]:
RegressionStep.run_from_dict(d)

Predicted 2182 values


In [26]:
model.register()

In [34]:
orca.list_steps()

['build_networks',
 'neighborhood_vars',
 'RegressionStep-20180211-190547',
 'RegressionStep-20180211-190813']

In [47]:
orca.run(['RegressionStep-20180211-190547'])

Running step 'RegressionStep-20180211-190547'
Predicted 2182 values
Time to execute step 'RegressionStep-20180211-190547': 0.06 s
Total time to execute iteration 1 with iteration value None: 0.06 s


In [ ]:
# Register a second one and make sure we can run them separately

In [29]:
tables = ['buildings', 'parcels']

model_expression = (
    "np.log1p(res_price_per_sqft) ~ "
        "parcel_acres + "
        "year_built")

# For prediction, specify destination column (if different from the dependent variable
# used for estimation), and how to reverse the left-hand-side transformation

out_fname = 'fitted_price'
ytransform = np.exp

In [30]:
model = RegressionStep(model_expression, tables, 
                       out_fname=out_fname, ytransform=ytransform)

In [31]:
model.fit()

                                 OLS Regression Results                                 
Dep. Variable:     np.log1p(res_price_per_sqft)   R-squared:                       0.395
Model:                                      OLS   Adj. R-squared:                  0.395
Method:                           Least Squares   F-statistic:                     712.3
Date:                          Sun, 11 Feb 2018   Prob (F-statistic):          9.49e-239
Time:                                  19:08:13   Log-Likelihood:                -2868.1
No. Observations:                          2182   AIC:                             5742.
Df Residuals:                              2179   BIC:                             5759.
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

In [44]:
model.register()

In [23]:
model.run()

Predicted 2182 values


In [48]:
df = orca.get_table('buildings').to_frame(['res_price_per_sqft', 'fitted_price'])
df.loc[df.fitted_price > 0].describe()

,res_price_per_sqft,fitted_price
count,2182.000000,2182.000000
mean,243.849936,206.818870
std,162.403472,137.454712
min,0.974158,0.007513
25%,128.734794,89.025805
50%,228.539201,191.793083
75%,365.494447,278.324228
max,1125.000000,1484.329439


In [41]:
orca.get_table('buildings').to_frame().describe()

,parcel_id,development_type_id,improvement_value,residential_units,non_residential_sqft,stories,year_built,residential_sqft,res_price_per_sqft,fitted_price,node_id
count,5.917200e+04,59172.000000,5.917200e+04,59172.000000,5.917200e+04,59172.000000,59172.000000,5.917200e+04,59172.000000,59172.000000,2182.000000
mean,1.824953e+06,19.335733,5.938045e+05,3.662661,8.495205e+03,1.081170,1979.705486,1.303178e+04,214.760125,7.626559,44284.095325
std,2.138605e+06,0.663925,2.823604e+06,13.734032,4.363458e+04,0.381794,20.675804,4.544811e+04,317.965487,47.070047,4104.251504
min,2.900000e+01,19.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,1913.000000,5.000000e+01,0.000220,0.000000,36360.000000
25%,2.894475e+05,19.000000,1.071312e+05,1.000000,0.000000e+00,1.000000,1965.000000,1.472000e+03,133.098672,0.000000,41131.000000
50%,5.873640e+05,19.000000,1.750000e+05,1.000000,0.000000e+00,1.000000,1981.000000,2.122000e+03,191.676955,0.000000,43936.500000
75%,5.032626e+06,19.000000,3.022590e+05,1.000000,0.000000e+00,1.000000,1998.000000,3.601000e+03,258.776213,0.000000,47438.000000
max,5.293696e+06,21.000000,1.800000e+08,679.000000,1.197079e+06,29.000000,2012.000000,1.197079e+06,21770.833333,1484.329439,52513.000000


In [25]:
# Add prediction components
# Register and save
# Delete
# Load from disk
